In [2]:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_anthropic

In [4]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Verify the API key is loaded
if not os.environ.get("ANTHROPIC_API_KEY"):
    raise ValueError("ANTHROPIC_API_KEY not found in environment variables")

In [3]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [5]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")


def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

In [6]:
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()

In [7]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

Assistant: I'm not sure what you're responding to with "Yeah." Is there something specific you'd like to talk about or ask? I'm here to help if you have any questions or topics you want to discuss!
User: What do you know about LangGraph?
Assistant: LangGraph is a framework for building stateful, multi-agent applications using large language models (LLMs). It builds on top of LangChain and implements principles from the Actor framework to enable creation of complex, stateful AI agents and workflows. LangGraph allows developers to define agents with memory and state that can interact with each other and external tools/APIs. It's designed to facilitate building more sophisticated AI systems and applications that go beyond simple question-answering.

Some key features and concepts of LangGraph include:

1. Stateful agents: Agents can maintain state and memory across interactions.

2. Multi-agent systems: Allows creation of systems with multiple interacting agents.

3. Workflow management: 